In [91]:
from tia.bbg import LocalTerminal
import tia.bbg.datamgr as dm
import pandas as pd
import numpy as np
import os
import sys
from pandas.tseries.offsets import BDay, Day
from datetime import datetime, date, time
import pyodbc

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=bvinf05;DATABASE=QAI;UID=qai_user;PWD=qai_user123')
cursor = cnxn.cursor()

# liquidity parameters
threshold = 300 * 10**6
adtv_days = 3.0
const_num = 30.0
adtv = (threshold / const_num) * (1 / adtv_days)

ws_us = pd.read_csv('Q:\\Index\\Reuters\\Mapping\\worldscope.us.mapping.csv')
ws_exus = pd.read_csv('Q:\\Index\\Reuters\\Mapping\\worldscope.mapping.csv')
ws = ws_us.append(ws_exus)

bbg = pd.read_csv('Q:\\Index\\Stoxx\\bbg data\\SD3P Index as of Feb 29 20161.csv')
bbg.columns = ['bb_ticker','name','weight','shares','price','id_sedol','id_isin','mcap_usd','security_typ']

my_dir = 'Q:\\Index\\Stoxx\\review reports'
os.chdir(my_dir)

xlsx = pd.ExcelFile('Q:\\Index\\Stoxx\\review reports\\qr_sxxp_20160223.xls')
univ = xlsx.parse(0, header=2, parse_cols=9, skiprows=1)
univ.to_csv('qr_sxxp.csv')

# read sxxp proforma files

univ = pd.read_csv('Q:\\Index\\Stoxx\\review reports\\qr_sxxp_20160223.csv')

# get withholding taxes

xlsx = pd.ExcelFile('Q:\\Index\\Stoxx\\withholding tax\\20160202 withholding_taxes.xls')
tax = xlsx.parse(0, header=0)
#print tax.head()

# get latest selection list

xlsx = pd.ExcelFile('Q:\\Index\\Stoxx\\selection lists\\sl_sd3p_201512.xls')
sl = xlsx.parse(0, header=4)
#print sl.head()

univ['ISIN'][univ['ISIN']=='JE00B2R84W06'] = 'JE00BD9WR069'
univ['Sedol'][univ['Sedol']=='B2R84W0'] = 'BD9WR06'

univ = univ.ix[:,['RIC','ISIN','Sedol','Company Name','Country','Currency']]
print univ.head()

df = pd.merge(univ, sl.ix[:,['ISIN','Company Name','Component','Rank(Final)','Dividend Yield','Country Dividend Yield','Regional Dividend Yield']], on='ISIN',how='outer')
df = pd.merge(df, tax.ix[:,['Code','WHT']], left_on='Country',right_on='Code',how='left')
df = pd.merge(df, bbg.ix[:,['id_isin','weight','shares']], left_on='ISIN',right_on='id_isin',how='outer')

df['cur_flag'] = np.where(pd.isnull(df['weight']), 0, 1)

df.to_csv('df.csv')

id_gb = ws.groupby('Id').first().reset_index()
isin = ws.groupby('Isin').first().reset_index()
id_final = df['ISIN'].map(isin.set_index('Isin').Id)

df['id_final'] = id_final
df['sedol_lst'] = df['id_final'].map(id_gb.set_index('Id').Sedol)
df['isin_lst'] = df['id_final'].map(id_gb.set_index('Id').Isin)

df.head()

df = df.fillna(0)
df = df[(df['id_final']!=0) | (df['cur_flag']==1)]

print df.head()
print df['cur_flag'].sum()

df.to_csv('df.csv')
my_df = df['id_final'].tolist()


########################################## GLOBAL ##################################################

# FUNDAMENTAL DATA AS OF CUTOFF


#sql = "SELECT * \
sql = "SELECT MS.SecCode, MS.Id, val.InfoCode, ctry.region, val.MarketDate, n.numshrs, h.FreeFloatPct, div.ISOCurrCode, x.PriceUnit \
    FROM GSecMstrX MS JOIN GSecMapX MP ON MS.SecCode = MP.SecCode AND MP.VenType = 33 \
    JOIN Ds2CtryQtInfo ctry ON ctry.InfoCode = MP.VenCode \
    JOIN Ds2PrimQtPrc val ON val.InfoCode= MP.VenCode \
    JOIN DS2EXCHQTINFO x ON x.INFOCODE = MP.VenCode AND x.INFOCODE=ctry.INFOCODE \
    JOIN DS2NUMSHARES n ON n.INFOCODE = x.INFOCODE AND n.EVENTDATE= (SELECT MAX(EVENTDATE) FROM DS2NUMSHARES WHERE INFOCODE=val.INFOCODE AND EVENTDATE<=val.MARKETDATE) \
    JOIN DS2SHAREHLDGS h ON h.INFOCODE = x.INFOCODE AND H.VALDATE= (SELECT MAX(VALDATE) FROM DS2SHAREHLDGS WHERE INFOCODE=val.INFOCODE AND VALDATE<=val.MARKETDATE) \
    JOIN DS2DIV div ON div.INFOCODE = x.INFOCODE AND div.EVENTNUM = (SELECT MAX(EVENTNUM) FROM DS2DIV WHERE INFOCODE=val.INFOCODE) \
    AND val.MarketDate = '2016-02-29' AND ctry.IsPrimQt=1 AND x.IsPrimExchQt='Y' \
    AND (MS.Id in (%s)" % ("'"+"', '".join(my_id)+"')") 

fifnos2 = pd.read_sql_query(sql, cnxn)
fifnos2.head()
fifnos2.to_csv('exus_prices_fx.csv', ignore_index=True)

fifnos2[fifnos2['Id']=='@FORTU12']

# PRICE AND VOLUME

sql = "SELECT MS.SecCode, MS.Id, val.InfoCode, ctry.region, ctry.TypeCode, val.MarketDate, fc.FROMCURRCODE, fx.MidRate,val.Close_,val.Volume, val.Close_/fx.MidRate as Price_Adj, dps.DPS \
    FROM GSecMstrX MS JOIN GSecMapX MP ON MS.SecCode = MP.SecCode AND MP.VenType = 33 \
    JOIN Ds2CtryQtInfo ctry ON ctry.InfoCode = MP.VenCode \
    JOIN Ds2xRef ref on ctry.TypeCode = ref.Code \
    JOIN Ds2PrimQtPrc val ON val.InfoCode= MP.VenCode \
    JOIN DS2FXCode fc ON fc.FROMCURRCODE = val.ISOCURRCODE AND fc.RATETYPECODE='SPOT' AND fc.TOCURRCODE='GBP' \
    JOIN DS2FXRate fx ON fx.EXRATEINTCODE = fc.EXRATEINTCODE AND fx.EXRATEDATE= val.MARKETDATE \
    JOIN DS2EXCHQTINFO x ON x.INFOCODE = MP.VenCode AND x.INFOCODE=ctry.INFOCODE \
    JOIN DS2DPS dps ON dps.INFOCODE = x.INFOCODE AND dps.EVENTDATE = (SELECT MAX(EVENTDATE) FROM DS2dps WHERE INFOCODE=val.INFOCODE AND EVENTDATE<=val.MARKETDATE) \
    AND val.MarketDate >= '2015-12-01' AND val.MarketDate <= '2016-02-29' AND ctry.IsPrimQt=1 AND x.IsPrimExchQt='Y' \
    AND (MS.Id in (%s)" % ("'"+"', '".join(my_id)+"')") 

px2 = pd.read_sql_query(sql, cnxn)
px2.head()

px2.to_csv('exus_prices_fx.csv', ignore_index=True)

# ADJ FACTORS

sql = "SELECT MS.SecCode, MS.Id, val.* \
    FROM GSecMstrX MS JOIN GSecMapX MP ON MS.SecCode = MP.SecCode AND MP.VenType = 33 \
    JOIN Ds2CtryQtInfo ctry ON ctry.InfoCode = MP.VenCode \
    JOIN DS2ADJ val ON val.InfoCode= MP.VenCode \
    WHERE ctry.IsPrimQt=1 \
    AND (MS.Id in (%s)" % ("'"+"', '".join(my_id)+"')") 
    
adj2 = pd.read_sql_query(sql, cnxn)

fifnos2['PriceUnit_Conv'] = fifnos2['PriceUnit'].str.replace('E','')
fifnos2['PriceUnit_Conv'] = 10**(fifnos2['PriceUnit_Conv'].astype(float))

end_date = '20801231'
end_date_pd = pd.to_datetime(end_date, format='%Y%m%d')
adj2['EndAdjDate'] = adj2['EndAdjDate'].fillna(end_date_pd)

px2['USDGBP'] = px2['MarketDate'].map(fx.set_index('EXRATEDATE').MidRate)
fifnos_dummy2 = fifnos2.groupby('Id').first().reset_index()
px2['PriceUnit_Conv'] = px2['Id'].map(fifnos_dummy2.set_index('Id').PriceUnit_Conv)


px2['Price'] = px2['Close_'] * px2['PriceUnit_Conv']
px2['Price_USD'] = px2['Price_Adj'] / px2['USDGBP'] * px2['PriceUnit_Conv']
px2['Turnover_USD'] = px2['Price_USD'] * px2['Volume']
px2['CumAdjFactor'] = np.piecewise(np.zeros(len(px2)), [(px2.MarketDate.values >= AdjDate)&(px2.MarketDate.values <= EndAdjDate)&(px2.Id.values == Id) for AdjDate, EndAdjDate, Id in zip(adj2.AdjDate.values, adj2.EndAdjDate.values, adj2.Id.values)], adj2.CumAdjFactor.values)
px2['Volume_Adj'] = px2['Volume'] * px2['CumAdjFactor']

px2.head()

turnover_3m_avg2 = px2.groupby('Id')['Turnover_USD'].mean().reset_index()
volume_adj_3m_avg2 = px2.groupby('Id')['Volume_Adj'].mean().reset_index()

univ2 = px2[px2['MarketDate']=='2016-02-29']
univ2 = pd.merge(univ2, fifnos2.ix[:,['Id','numshrs','FreeFloatPct','ISOCurrCode','PriceUnit']], on='Id',how='left')
univ2 = univ2.drop('Turnover_USD',1)
univ2 = pd.merge(univ2, turnover_3m_avg2.ix[:,['Id','Turnover_USD']], on='Id',how='left')
univ2 = univ2.drop('Volume_Adj',1)
univ2 = pd.merge(univ2, volume_adj_3m_avg2.ix[:,['Id','Volume_Adj']], on='Id',how='left')

univ2['DivYield'] = univ2['DPS'] / univ2['Price']

univ2.head()

univ = univ.append(univ2)

# PERFORM SQL

sql1 = "SELECT MS.Isin, MS.Sedol, MS.Sedol2, MS.Id, MS.Name, MS.Country, MS.Type_, VAL.*, Wsitem.NAME \
    FROM GSecMstrX MS JOIN GSecMapX MP ON MS.SecCode = MP.SecCode AND MP.VenType = 25 \
    JOIN WSInfo WS1 ON WS1.Code = MP.VenCode \
    JOIN WSndata VAL ON VAL.Code = MP.VenCode \
    JOIN Wsitem ON VAL.Item = Wsitem.Number \
    WHERE VAL.Item = 18264 \
    AND (MS.Id in (%s)" % ("'"+"', '".join(my_id)+"')")

sql1_dt = "SELECT MS.Isin, MS.Sedol, MS.Sedol2, MS.Id, MS.Name, MS.Country, MS.Type_, VAL2.*, Wsitem.Name\
    FROM GSecMstrX MS JOIN GSecMapX MP ON MS.SecCode = MP.SecCode AND MP.VenType = 25 \
    JOIN WSInfo WS1 ON WS1.Code = MP.VenCode \
    JOIN WSddata VAL2 ON VAL2.Code = MP.VenCode \
    JOIN Wsitem ON VAL2.Item = Wsitem.Number \
    WHERE VAL2.Item = 5905 \
    AND (MS.Id in (%s)" % ("'"+"', '".join(my_id)+"')")

sql2 = "SELECT MS.Isin, MS.Sedol, MS.Sedol2, MS.Id, MS.Name, MS.Country, MS.Type_, VAL.*, Wsitem.NAME \
    FROM SecMstrX MS JOIN SecMapX MP ON MS.SecCode = MP.SecCode AND MP.VenType = 25 \
    JOIN WSInfo WS1 ON WS1.Code = MP.VenCode \
    JOIN WSndata VAL ON VAL.Code = MP.VenCode \
    JOIN Wsitem ON VAL.Item = Wsitem.Number \
    WHERE VAL.Item = 18264 \
    AND (MS.Id in (%s)" % ("'"+"', '".join(my_id)+"')")

sql2_dt = "SELECT MS.Isin, MS.Sedol, MS.Sedol2, MS.Id, MS.Name, MS.Country, MS.Type_, VAL2.*, Wsitem.Name\
    FROM SecMstrX MS JOIN SecMapX MP ON MS.SecCode = MP.SecCode AND MP.VenType = 25 \
    JOIN WSInfo WS1 ON WS1.Code = MP.VenCode \
    JOIN WSddata VAL2 ON VAL2.Code = MP.VenCode \
    JOIN Wsitem ON VAL2.Item = Wsitem.Number \
    WHERE VAL2.Item = 5905 \
    AND (MS.Id in (%s)" % ("'"+"', '".join(my_id)+"')")

sql3 = "SELECT MS.Isin, MS.Sedol, MS.Sedol2, MS.Id, MS.Name, MS.Country, MS.Type_, VAL.*, Wsitem.NAME \
    FROM GSecMstrX MS JOIN GSecMapX MP ON MS.SecCode = MP.SecCode AND MP.VenType = 25 \
    JOIN WSInfo WS1 ON WS1.Code = MP.VenCode \
    JOIN WSndata VAL ON VAL.Code = MP.VenCode \
    JOIN Wsitem ON VAL.Item = Wsitem.Number \
    WHERE VAL.Item = 18266 \
    AND (MS.Id in (%s)" % ("'"+"', '".join(my_id)+"')")

sql3_dt = "SELECT MS.Isin, MS.Sedol, MS.Sedol2, MS.Id, MS.Name, MS.Country, MS.Type_, VAL2.*, Wsitem.Name\
    FROM GSecMstrX MS JOIN GSecMapX MP ON MS.SecCode = MP.SecCode AND MP.VenType = 25 \
    JOIN WSInfo WS1 ON WS1.Code = MP.VenCode \
    JOIN WSddata VAL2 ON VAL2.Code = MP.VenCode \
    JOIN Wsitem ON VAL2.Item = Wsitem.Number \
    WHERE (VAL2.Item = 5923 OR VAL2.Item = 5924 OR VAL2.Item = 5925 OR VAL2.Item = 5926) \
    AND (MS.Id in (%s)" % ("'"+"', '".join(my_id)+"')")

sql4 = "SELECT MS.Isin, MS.Sedol, MS.Sedol2, MS.Id, MS.Name, MS.Country, MS.Type_, VAL.*, Wsitem.NAME \
    FROM SecMstrX MS JOIN SecMapX MP ON MS.SecCode = MP.SecCode AND MP.VenType = 25 \
    JOIN WSInfo WS1 ON WS1.Code = MP.VenCode \
    JOIN WSndata VAL ON VAL.Code = MP.VenCode \
    JOIN Wsitem ON VAL.Item = Wsitem.Number \
    WHERE VAL.Item = 18266 \
    AND (MS.Id in (%s)" % ("'"+"', '".join(my_id)+"')")

sql4_dt = "SELECT MS.Isin, MS.Sedol, MS.Sedol2, MS.Id, MS.Name, MS.Country, MS.Type_, VAL2.*, Wsitem.Name\
    FROM SecMstrX MS JOIN SecMapX MP ON MS.SecCode = MP.SecCode AND MP.VenType = 25 \
    JOIN WSInfo WS1 ON WS1.Code = MP.VenCode \
    JOIN WSddata VAL2 ON VAL2.Code = MP.VenCode \
    JOIN Wsitem ON VAL2.Item = Wsitem.Number \
    WHERE (VAL2.Item = 5923 OR VAL2.Item = 5924 OR VAL2.Item = 5925 OR VAL2.Item = 5926) \
    AND (MS.Id in (%s)" % ("'"+"', '".join(my_id)+"')")

eps_exus = pd.read_sql_query(sql1, cnxn)
#eps_us = pd.read_sql_query(sql2, cnxn)
eps_exus_dt = pd.read_sql_query(sql1_dt, cnxn)
#eps_us_dt = pd.read_sql_query(sql2_dt, cnxn)

dps_exus = pd.read_sql_query(sql3, cnxn)
#dps_us = pd.read_sql_query(sql4, cnxn)
dps_exus_dt = pd.read_sql_query(sql3_dt, cnxn)
#dps_us_dt = pd.read_sql_query(sql4_dt, cnxn)


df2 = pd.merge(df, univ.ix[:,['Id','region','Price','Price_Adj','Price_USD','DPS','DivYield','CumAdjFactor','Turnover_USD','Volume_Adj','FreeFloatPct','numshrs','ISOCurrCode']], left_on='id_final',right_on='Id',how='left')

df2 = df2.fillna(0)

df2['final_fif'] = np.where(df2['fif']==0,df2['FreeFloatPct']/100,df2['fif'])
df2['final_nos'] = np.where(df2['nos']==0,df2['numshrs'] * 1000, df2['nos'])

df2['Full_Mcap_USD'] = df2['Price_USD'] * df2['final_nos']
df2['FF_Mcap_USD'] = df2['Full_Mcap_USD'] * df2['final_fif']

df2 = df2.drop('Id',1)

eps = eps_exus.append(eps_us) 
eps_dt = eps_exus_dt.append(eps_us_dt) 

eps['unique_id'] = eps.Id.str.cat(eps.Freq)
eps['unique_id'] = eps.unique_id.str.cat(eps.Year_.astype(str))
eps['unique_id'] = eps.unique_id.str.cat(eps.Seq.astype(str))

eps_dt['unique_id'] = eps_dt.Id.str.cat(eps_dt.Freq)
eps_dt['unique_id'] = eps_dt.unique_id.str.cat(eps_dt.Year_.astype(str))
eps_dt['unique_id'] = eps_dt.unique_id.str.cat(eps_dt.Seq.astype(str))

eps = pd.merge(eps, eps_dt.ix[:,['unique_id','Date_','Value_']], left_on = 'unique_id', right_on='unique_id', how='inner')
eps = eps.rename(columns={'Date__x' : 'Date_'})
eps = eps.rename(columns={'Value__x' : 'Value_'})
eps = eps.rename(columns={'Date__y' : 'DATE_'})
eps = eps.rename(columns={'Value__y' : 'VALUE_'})

eps['Value_'] = eps['Value_'].fillna(0)
#eps = eps[eps['Value_']!=0]
eps = eps.sort_values(by='DATE_',ascending=1)
eps.loc[:,'RptYear'] = (eps['VALUE_'].astype(str).str[:4]).astype(int)
eps_groupby = eps.groupby(['Isin','Id','Name','Country','RptYear'])['DATE_','VALUE_','Value_'].last().reset_index()
eps.to_csv(my_index + '_eps.csv',ignore_index=True)
eps_groupby.to_csv(my_index + 'eps_groupby.csv',ignore_index=True)

print len(eps['Isin'].unique())
print len(eps['Id'].unique())

eps_year5 = eps_groupby[eps_groupby['RptYear']==year5]
eps_year4 = eps_groupby[eps_groupby['RptYear']==year4]
eps_year3 = eps_groupby[eps_groupby['RptYear']==year3]
eps_year2 = eps_groupby[eps_groupby['RptYear']==year2]
eps_year1 = eps_groupby[eps_groupby['RptYear']==year1]
eps_year0 = eps_groupby[eps_groupby['RptYear']==year0]

df3 = pd.merge(eps_year5, eps_year4.ix[:,['Isin','Value_']], left_on='Isin', right_on='Isin', how='outer')
df3 = df3.rename(columns={'Value__x' : 'year5'})
df3 = df3.rename(columns={'Value__y' : 'year4'})
df3 = pd.merge(df3, eps_year3.ix[:,['Isin','Value_']], left_on='Isin', right_on='Isin', how='outer')
df3 = df3.rename(columns={'Value_' : 'year3'})
df3 = pd.merge(df3, eps_year2.ix[:,['Isin','Value_']], left_on='Isin', right_on='Isin', how='outer')
df3 = df3.rename(columns={'Value_' : 'year2'})
df3 = pd.merge(df3, eps_year1.ix[:,['Isin','Value_']], left_on='Isin', right_on='Isin', how='outer')
df3 = df3.rename(columns={'Value_' : 'year1'})
df3 = pd.merge(df3, eps_year0.ix[:,['Isin','Value_']], left_on='Isin', right_on='Isin', how='outer')
df3 = df3.rename(columns={'Value_' : 'year0'})

df3.head()
print len(df3)
df3.columns = ['Isin','Id','Name','Country','RptYear','PeriodDate','RptDate','eps_year5','eps_year4','eps_year3','eps_year2','eps_year1','eps_year0']
eps2 = df3.copy()


C:\Users\yong.joon.cho\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yong.joon.cho\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


       RIC          ISIN    Sedol  Company Name Country Currency
0   NESN.S  CH0038863350  7123870        NESTLE      CH      CHF
1   NOVN.S  CH0012005267  7103065      NOVARTIS      CH      CHF
2    ROG.S  CH0012032048  7110388  ROCHE HLDG P      CH      CHF
3   HSBA.L  GB0005405286   540528          HSBC      GB      GBP
4  TOTF.PA  FR0000120271  B15C557         TOTAL      FR      EUR
       RIC          ISIN    Sedol Company Name_x Country Currency  \
0   NESN.S  CH0038863350  7123870         NESTLE      CH      CHF   
1   NOVN.S  CH0012005267  7103065       NOVARTIS      CH      CHF   
2    ROG.S  CH0012032048  7110388   ROCHE HLDG P      CH      CHF   
3   HSBA.L  GB0005405286   540528           HSBC      GB      GBP   
4  TOTF.PA  FR0000120271  B15C557          TOTAL      FR      EUR   

  Company Name_y Component  Rank(Final)  Dividend Yield      ...       \
0              0         0          0.0         0.00000      ...        
1              0         0          0.0         0